# **RETO HACKATON**: *CIENCIA DE DATOS - CAIXA BANK*
*Carlos Cabruja - Data*

## Background

El IBEX 35 es el índice oficial de la bolsa española compuesto por las 35 empresas más negociadas del mercado. Este índice nos muestra en tiempo real si los precios en bolsa están subiendo o bajando, por lo que permite medir el comportamiento de este conjunto de acciones.

El IBEX35 sirve como punto de referencia para los inversores del mercado español. La rentabilidad de este índice es el objetivo a batir por los gestores.

Por lo tanto, la modelización de las dinámicas de este tipo de índices resultan esenciales para la toma de decisiones por parte de todas las entidades bursátiles.

## Reto

1. Desarrolla un modelo predictivo que permita predecir la variable target (si el precio de cierre del IBEX35 será superior o inferior al precio de cierre actual).

Para ello deberas entrenar tu modelo con los datos de training (si también se usan los tweets se sumaran 100 puntos) e introducir como input de tu modelo el dataset test_x para realizar las predicciones.

2. Crea un breve documento (máx. 2 páginas) o presentación (máx. 4 slides) explicando la solución que has empleado y porque la has empleado.

## Librerías

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import nltk
import datetime as dt

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

## Datos

In [ ]:
# cargamos los datasets a trabajar
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test_x.csv')
tweets = pd.read_csv('data/tweets_from2015_#Ibex35.csv')

In [ ]:
train # visualizamos los datos de entrenamiento

In [ ]:
test # visualizamos los datos de prueba

In [ ]:
tweets # visualizamos los datos de tweets

In [ ]:
train.info()

Habrá que hacer tratamiento de NULL en los datos y hacer que **Date** sea una variable DateTime.

In [ ]:
test.info()

In [ ]:
tweets.info()

Hacer **tweetdate** un dateTime para que pueda hacer merge con nuestro dataframe de train, y tratamiento de nulls

## Limpieza de datos

In [ ]:
# cuantos nulls tiene tweets?
tweets.isnull().sum()

In [ ]:
# filas con nulls en tweetDate
tweets[tweets.tweetDate.isnull()]

No hay nada, eliminamos....

In [ ]:
# eliminar filas con nulls en tweetDate
tweets = tweets.dropna(subset=['tweetDate'])

# filas con nulls en text
tweets[tweets.text.isnull()]

Tampoco son relevantes los tweets que no tienen fecha de publicación, ni texto de los tweets.

In [ ]:
# eliminar todos los nulls
tweets = tweets.dropna()
tweets.isnull().sum()

In [ ]:
# convertir Date a datetime YYYY-MM-DD
try:
    tweets['tweetDate'] = pd.to_datetime(tweets['tweetDate'])
except Exception as e:
    print(e)

Como **tweetdate** está contaminada con texto la vamos a tratar fila por fila. 

In [ ]:
for i in tweets['tweetDate']:
    lista_i = str(i).split(' ')
    if len(lista_i) != 6: # si no tiene 6 elementos (YYYY-MM-DD HH:MM:SS)
        # eliminar filas con fechas mal formateadas
        tweets = tweets.drop(tweets[tweets['tweetDate'] == i].index)

In [ ]:
# convertir Date a datetime YYYY-MM-DD
tweets['tweetDate'] = pd.to_datetime(tweets['tweetDate'])

In [ ]:
# renombrar tweetDate a Date
tweets = tweets.rename(columns={'tweetDate': 'Date'})

Y la columna handle no nos sirve ya que no daremos importancia a quién ha escrito el tweet.

In [ ]:
# eliminar handle de los tweets
tweets = tweets.drop(['handle'], axis=1)
tweets

Ahora creamos la columna sentimiento, con la libreria de Sentiment Analysis de NLTK

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer() # instanciamos el analizador

In [ ]:
tweets['sentiment'] = tweets['text'].apply(lambda x: sentiment.polarity_scores(x)['compound'])
tweets

In [ ]:
# eliminar text
tweets = tweets.drop(['text'], axis=1)

Ahora, a fin de hacer el merge con train, trataremos los datos para que sea solo una fecha.

Para los tweets hechos en una misma fecha pero distinta hora, se hará una diferencia entre los sentimientos

In [ ]:
# convertir Date a datetime YYYY-MM-DD
tweets['Date'] = pd.to_datetime(tweets['Date']).dt.date

In [ ]:
# eliminar Dates repetidos con el mismo sentimiento y fecha
tweets = tweets.drop_duplicates()

# hay alguna fecha repetida con distinto sentimiento?
tweets.groupby(['Date', 'sentiment']).count()

In [ ]:
# sacamos las fechas repetidas con distinto sentimiento
temp_df = tweets.groupby(['Date']).count()
lista_fechas = temp_df[temp_df['sentiment'] > 1].index

# restamos el indice de sentimiento en las fechas repetidas
for i in lista_fechas:
    # sacamos las filas repetidas
    temp_df = tweets[tweets['Date'] == i]
    print(temp_df)